In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.auto import tqdm, trange

import string
from bs4 import BeautifulSoup

import nltk
from nltk.stem import *
from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

## **Загрузка датасета**

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/диссертация/data/notes_final.csv')

<ipython-input-3-bdf4ba9049a9>:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/диссертация/data/notes_final.csv')


In [ ]:
data

,id,diary,text,date,sex,birth,age
0,693,3,<p>Сейчас бьет на башенных часах 12. Полночь. ...,1932-09-28,1.0,1890-05-25,42.0
1,694,3,"<p>Читал Шаляпина — воспоминания. Оказывается,...",1932-10-06,1.0,1890-05-25,42.0
2,695,3,<p>При социализме заводы суть храмы и центры ч...,1932-10-20,1.0,1890-05-25,42.0
3,696,3,<p>Завтра с утра — в Москву. Что-то меня там ж...,1932-10-30,1.0,1890-05-25,42.0
4,697,3,"<p>В Берлине, в полпредстве, оказался Горький ...",1932-11-01,1.0,1890-05-25,42.0
...,...,...,...,...,...,...,...
437847,785544,9939,"<p><strong>21 сентября (понедельник), 1970 год...",1970-09-21,1.0,1940-06-12,30.0
437848,785545,9939,"<p><strong>22 сентября (вторник), 1970 год</st...",1970-09-22,1.0,1940-06-12,30.0
437849,785546,9939,"<p><strong>23 сентября (среда), 1970 год</stro...",1970-09-23,1.0,1940-06-12,30.0
437850,785547,9939,"<p><strong>24 сентября (четверг), 1970 год</st...",1970-09-24,1.0,1940-06-12,30.0


## **Предобработка: удаление знаков препинания, тегов, чисел и т.д.**

In [ ]:
def remove_html_tags(text):
  """Функция для удаления HTML-тегов"""
  soup = BeautifulSoup(text, 'html.parser')
  return soup.get_text()

In [ ]:
def remove_punctuation(text):
  """Функция удаления знаков пунктуации"""
  return "".join([ch if ch not in string.punctuation else ' ' for ch in text])

def remove_numbers(text):
  """Функция удаления чисел"""
  return "".join([i if not i.isdigit() else ' ' for i in text])

def remove_multiple_spaces(text):
  """Функция удаления множественных пробелов"""
  return re.sub(r'\s+', ' ', text, flags=re.I)

In [ ]:
data['text'] = data['text'].apply(remove_html_tags)

<ipython-input-12-214274352668>:3: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, 'html.parser')


In [ ]:
prep_text = [remove_multiple_spaces(remove_numbers(remove_punctuation(text.lower()))) for text in tqdm(data['text'])]

  0%|          | 0/437852 [00:00<?, ?it/s]

In [ ]:
data['text_preprocessing'] = prep_text

In [ ]:
data.head(5)

,id,diary,text,date,sex,birth,age,text_preprocessing
0,693,3,Сейчас бьет на башенных часах 12. Полночь. Жен...,1932-09-28,1.0,1890-05-25,42.0,сейчас бьет на башенных часах полночь жена уже...
1,694,3,"Читал Шаляпина — воспоминания. Оказывается, ис...",1932-10-06,1.0,1890-05-25,42.0,читал шаляпина — воспоминания оказывается исто...
2,695,3,При социализме заводы суть храмы и центры чело...,1932-10-20,1.0,1890-05-25,42.0,при социализме заводы суть храмы и центры чело...
3,696,3,Завтра с утра — в Москву. Что-то меня там ждет...,1932-10-30,1.0,1890-05-25,42.0,завтра с утра — в москву что то меня там ждет ...
4,697,3,"В Берлине, в полпредстве, оказался Горький (пр...",1932-11-01,1.0,1890-05-25,42.0,в берлине в полпредстве оказался горький проез...


In [ ]:
data['text_preprocessing'][5]

'в берлин приехал я с гр ив петровским нас поместили с ним в одной квартире в посольстве а наутро мы поехали в москву через варшаву в поезде после польской границы стало грязнее обслуживающий персонал небрежнее и бестолковее будто бы все житейское начало понемногу терять смысл это ужасное отличие европейца от жителя польско русской равнины последний как будто не совсем твердо знает за каким собственно делом он появился на свет каково его место среди других европеец же с семнадцати лет это знает и знает когда он умрет и при каком капитале приехали рано утром в варшаву она всегда провинция полпредство выглядит учреждением с фанерными перегородками провели нас в комнату очень холодную беседовал с антоновым овсеенко он рассказал о кознях против меня м п советничек а по существу сепаратист но не вследствие национальных чувств а вследствие мелкобуржуазной породы не способной объять нужды и трудности государства протяжением громадном — от атлантики до тихого и от белого до черного — и ищущий 

## **Предобработка: удаление стоп-слов и лемматизация**

In [ ]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
mystem = Mystem()

russian_stopwords = stopwords.words("russian")
russian_stopwords.extend(['…', '«', '»', '...', '—', ''])

def lemmatize_text(text):
    tokens = mystem.lemmatize(text)
    tokens = [token for token in tokens if token not in russian_stopwords and token != " "]
    text = " ".join(tokens)
    return text

In [ ]:
for index, row in tqdm(data.iterrows(), total=len(data)):
    data.at[index, 'text_preprocessing'] = lemmatize_text(row['text_preprocessing'])

  0%|          | 0/437852 [00:00<?, ?it/s]

In [ ]:
data.head()

,id,diary,text,date,sex,birth,age,text_preprocessing
0,693,3,Сейчас бьет на башенных часах 12. Полночь. Жен...,1932-09-28,1.0,1890-05-25,42.0,бить башенный часы полночь жена спать ребенок ...
1,694,3,"Читал Шаляпина — воспоминания. Оказывается, ис...",1932-10-06,1.0,1890-05-25,42.0,читать шаляпин — воспоминание оказываться ис...
2,695,3,При социализме заводы суть храмы и центры чело...,1932-10-20,1.0,1890-05-25,42.0,социализм завод храм центр человеческий житие ...
3,696,3,Завтра с утра — в Москву. Что-то меня там ждет...,1932-10-30,1.0,1890-05-25,42.0,завтра утро — москва ждать каково настроение...
4,697,3,"В Берлине, в полпредстве, оказался Горький (пр...",1932-11-01,1.0,1890-05-25,42.0,берлин полпредство оказываться горький проезд ...


In [ ]:
data.to_csv('data_preprocessing.csv')

In [ ]:
data['text'][3]

'Завтра с утра — в Москву. Что-то меня там ждет, каково-то там настроение… Как широко пользуются свободой клеветы пакостники… Работа литературная мною будет поставлена на первое место, ведь жить осталось немного. Каждый человек, в том числе и я,\xa0— талант, а талант — хрустальная чаша с нектаром. Нужно, чтобы чаша была в движении и расплескивала бы свой нектар. Моя чаша мало, очень мало была в движении. В литературе я понимаю, вероятно, меньше, чем понимал до революции. Жизнь мобилизовала мои силы на другую работу. Теперь я снова должен тянуться к искусству'

In [ ]:
data['text_preprocessing'][3]

'завтра утро  —  москва ждать каково настроение широко пользоваться свобода клевета пакостник работа литературный поставлять первый место жить оставаться немного каждый человек число  —  талант талант  —  хрустальный чаша нектар нужно чаша движение расплескивать свой нектар чаша мало очень мало движение литература понимать вероятно мало понимать революция жизнь мобилизовать сила работа снова должный тянуться искусство \n'

## **Работа с текстами на другом языке**



In [ ]:
data = pd.read_csv('/content/drive/MyDrive/диссертация/data/data_preprocessing.csv')

<ipython-input-3-77f8dacdfbe5>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/диссертация/data/data_preprocessing.csv')


### —> **Детектирование языка текста**

In [ ]:
data.head(1)

,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing
0,0,693,3,Сейчас бьет на башенных часах 12. Полночь. Жен...,1932-09-28,1.0,1890-05-25,42.0,бить башенный часы полночь жена спать ребенок ...


In [ ]:
! pip install langdetect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 7.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993227 sha256=292be569b0c641c9e0d6cd7ca7fc95f7d3850915901f6e35a59c0e04050e181c
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [ ]:
from langdetect import detect, LangDetectException

def detect_language(text):
    try:
        if pd.isna(text):
            return 'undefined', text  # Если текст является NaN, возвращаем 'undefined'
        return detect(text)
    except LangDetectException:
        return 'error'  # Если язык не удается определить

In [ ]:
data.dropna()

In [ ]:
# Создаем пустой список для хранения результатов
languages = []

# Используем цикл для обработки каждого текста
for text in tqdm(data['text_preprocessing']):
    language = detect_language(text)
    languages.append(language)

  0%|          | 0/437852 [00:00<?, ?it/s]

In [ ]:
data['language'] = languages

In [ ]:
# Фильтрация записей, язык которых отличен от русского
non_russian_texts = data[data['language'] != 'ru']
print(f"Количество текстов на не русских языках: {non_russian_texts.shape[0]}")

# Вывод количества текстов на каждом языке, не считая русский
print(non_russian_texts['language'].value_counts())

Количество текстов на не русских языках: 11877
language
uk                  8267
bg                  2247
mk                   738
fr                   224
error                119
et                    63
en                    61
de                    48
af                    19
it                    19
no                     9
(undefined, nan)       8
ro                     7
ca                     7
sl                     6
pl                     5
so                     5
nl                     4
sv                     4
vi                     3
da                     3
hr                     2
tl                     2
el                     2
cy                     1
id                     1
pt                     1
fi                     1
sk                     1
Name: count, dtype: int64


In [ ]:
data.to_csv('data_preprocessing_with_language.csv')

### —> **Работа с текстами другого языка**

In [43]:
data = pd.read_csv('/content/drive/MyDrive/диссертация/data/data_preprocessing_with_language.csv')

<ipython-input-43-4524c94e25f0>:1: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('/content/drive/MyDrive/диссертация/data/data_preprocessing_with_language.csv')


In [44]:
data['language'].unique()

array(['ru', 'bg', 'uk', 'de', 'mk', 'en', 'et', 'fr', 'error', 'pl',
       'ro', 'it', "('undefined', nan)", 'ca', 'vi', 'sl', 'no', 'da',
       'nl', 'cy', 'hr', 'af', 'id', 'tl', 'sv', 'el', 'so', 'pt', 'fi',
       'sk'], dtype=object)

In [6]:
# Тексты на немецком языке
de_data = data[data['language'] == 'de']
de_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
1679,1679,1679,2876,10,"\nDer Mensch ist nicht geboren, frei zu sein,\...",1920-12-16,1.0,1880-11-28,40.0,der mensch ist nicht geboren frei zu sein und ...,de
1719,1719,1719,2916,10,"Заключительные слова «Золота Рейна», вложенные...",1921-04-17,1.0,1880-11-28,41.0,заключительный слово « золото рейн » вкладыв...,de
89173,89173,89173,127217,122,Spiegel ist zerbrochen? Oder noch nicht? Was w...,1920-06-08,1.0,1891-03-24,29.0,spiegel ist zerbrochen oder noch nicht was wei...,de
89657,89657,89657,127701,122,Ленинград. Сводка: \n\n«So bin ich aus mir ...,1945-12-31,1.0,1891-03-24,54.0,ленинград сводка « so bin ich aus mir selbst ...,de
115127,115127,115127,177195,840,"Троицын день. Pfingsten, das herrliche Fest. Д...",1874-05-31,1.0,1815-09-22,59.0,троицын день pfingsten das herrliche fest herr...,de


In [20]:
de_data.shape

(48, 11)

In [27]:
# Тексты на английском языке
en_data = data[data['language'] == 'en']
en_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
14480,14480,14480,18742,53,"Болею, а поэтому каждый день слушаю VOA Breakf...",1978-03-29,1.0,1936-03-09,42.0,болеть поэтому каждый день слушать voa breakfa...,en
30455,30455,30455,42995,184,Возьму с собой на всякий случай «Social Scienc...,1969-06-19,1.0,1923-09-14,46.0,взять всякий случай « social sciences to daу ...,en
30849,30849,30849,43389,184,Надпись на подарке Лу Розенблюма:\nPhoto: Cosm...,1974-12-14,1.0,1923-09-14,51.0,надпись подарок лу розенблюм photo cosmic graf...,en
30866,30866,30866,43406,184,Получил вчера от Стивена Левина телеграмму: Bo...,1975-02-19,1.0,1923-09-14,52.0,получать вчера стивен левин телеграмма book at...,en
30891,30891,30891,43431,184,Вчера дал Мак-Гиллу и де Бари следующую телегр...,1975-04-27,1.0,1923-09-14,52.0,вчера давать мак гилл де бари следующий телегр...,en


In [28]:
en_data.shape

(61, 11)

In [31]:
# Тексты на французском языке
fr_data = data[data['language'] == 'fr']
fr_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
27146,27146,27146,38416,131,"Я увлекаюсь, читая «Vie et correspondance de T...",1919-09-12,1.0,1873-06-30,46.0,увлекаться читать « vie et correspondance de ...,fr
31391,31391,31391,44233,210,Вчера был у Оболенской. Elle m’a dit de faire ...,1940-07-03,1.0,1925-02-01,15.0,вчера оболенская elle m ’ a dit de faire une n...,fr
31410,31410,31410,44252,210,Hier j’ai passé une bonne fin de soirée. On a ...,1940-07-29,1.0,1925-02-01,15.0,hier j ’ ai passé une bonne fin de soirée on a...,fr
31441,31441,31441,44283,210,Nous avons déménagé. Nous vivons très bien (cô...,1940-09-29,1.0,1925-02-01,15.0,nous avons déménagé nous vivons très bien côté...,fr
31467,31467,31467,44309,210,"Ô joie! J’écoute en ce moment un excellent, jo...",1940-11-12,1.0,1925-02-01,15.0,ô joie j ’ écoute en ce moment un excellent jo...,fr


In [32]:
fr_data.shape

(224, 11)

In [33]:
# Тексты, которые не были определены
error_data = data[data['language'] == 'error']
error_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
28687,28687,28687,40503,238,то же,1945-04-06,1.0,1922-01-01,23.0,\n,error
28708,28708,28708,40524,238,2921,1945-05-10,1.0,1922-01-01,23.0,\n,error
28709,28709,28709,40525,238,2921,1945-05-11,1.0,1922-01-01,23.0,\n,error
28710,28710,28710,40526,238,2921,1945-05-12,1.0,1922-01-01,23.0,\n,error
66954,66954,66954,88588,351,Ничего.,1990-06-12,1.0,1970-01-01,20.0,\n,error


In [34]:
error_data.shape

(119, 11)

In [35]:
# Тексты на польском языке
pl_data = data[data['language'] == 'pl']
pl_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
110762,110762,110762,172183,811,\nLekarz Karol Nowicki \nPawel Radziejowski ...,1857-08-31,1.0,1814-03-09,43.0,lekarz karol nowicki pawel radziejowski tytus ...,pl
110763,110763,110763,172184,811,Krasomówstwo niewielu otrzymato w udziałe; mni...,1857-09-01,1.0,1814-03-09,43.0,krasomówstwo niewielu otrzymato w udziałe mnie...,pl
117458,117458,117458,181323,859,"Суббота. « Gazza ladra». Cow.""",1826-11-20,1.0,1783-02-09,43.0,суббота « gazza ladra » cow \n,pl
242166,242166,242166,406715,1847,Wien.,1900-08-25,1.0,1860-01-01,40.0,wien \n,pl
242526,242526,242526,407077,1847,Wien.,1910-01-14,1.0,1860-01-01,50.0,wien \n,pl


In [36]:
pl_data.shape

(5, 11)

In [37]:
# Тексты на молдавском языке
ro_data = data[data['language'] == 'ro']
ro_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
118717,118717,118717,183346,155,Получил у консула certificate de coutume.,1914-04-04,1.0,1853-07-27,61.0,получать консул certificate de coutume \n,ro
175873,175873,175873,282855,859,Суббота. С великим князем в церкви dei Frari. ...,1838-11-24,1.0,1783-02-09,55.0,суббота великий князь церковь dei frari s roch...,ro
199797,199797,199797,324266,963,30. Ditile Электр[?] Dr,1904-02-12,1.0,1832-01-01,72.0,ditile электр dr \n,ro
242045,242045,242045,406594,1847,Lucerne.,1897-06-10,1.0,1860-01-01,37.0,lucerne \n,ro
282912,282912,282912,486110,3885,(субб): ordinarii,1813-02-20,1.0,1776-01-24,37.0,субб ordinarii \n,ro


In [38]:
ro_data.shape

(7, 11)

In [39]:
# Тексты на итальянском языке
it_data = data[data['language'] == 'it']
it_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
133089,133089,133089,206716,193,Рим. Это заметка из какой-то итальянской газет...,1980-04-15,1.0,1932-04-04,48.0,рим это заметка итальянский газета марта г « ...,it
144922,144922,144922,227195,19,"Villa Jeannette, Grasse, a — m.",1940-01-01,1.0,1870-10-22,70.0,villa jeannette grasse a — m \n,it
150695,150695,150695,242957,1315,Silentium.,1961-11-19,1.0,1938-10-24,23.0,silentium \n,it
171399,171399,171399,275205,122,«Ne avrai la forza?[»] Io rispondevo a me stes...,1912-12-31,1.0,1891-03-24,21.0,ne avrai la forza » io rispondevo a me stess...,it
199808,199808,199808,324278,963,13. Ditil.,1904-02-26,1.0,1832-01-01,72.0,ditil \n,it


In [40]:
it_data.shape

(19, 11)

In [41]:
# Nan
undefined_data = data[data['language'] == "('undefined', nan)"]
undefined_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
142459,142459,142459,223825,979,NaN,1813-02-27,1.0,1784-09-21,29.0,NaN,"('undefined', nan)"
142461,142461,142461,223827,979,NaN,1813-03-13,1.0,1784-09-21,29.0,NaN,"('undefined', nan)"
231442,231442,231442,388329,2331,NaN,1923-04-05,0.0,1913-12-01,10.0,NaN,"('undefined', nan)"
231443,231443,231443,388330,2331,NaN,1923-04-06,0.0,1913-12-01,10.0,NaN,"('undefined', nan)"
231455,231455,231455,388342,2331,NaN,1923-04-18,0.0,1913-12-01,10.0,NaN,"('undefined', nan)"


In [42]:
undefined_data.shape

(8, 11)

In [43]:
# Тексты на каталанском языке
ca_data = data[data['language'] == 'ca']
ca_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
148400,148400,148400,239593,672,Niagara Falls.,1967-06-24,1.0,1933-06-30,34.0,niagara falls \n,ca
199679,199679,199679,324147,963,24. Мы приех[али] в Paris Hotel Continental.,1903-09-06,1.0,1832-01-01,71.0,приех али paris hotel continental \n,ca
240841,240841,240841,404953,879,3.IX.,1921-09-03,1.0,1877-07-06,44.0,ix \n,ca
252720,252720,252720,427165,840,Maxima eorum temporum felicitas ubi quae senti...,1881-09-05,1.0,1815-09-22,66.0,maxima eorum temporum felicitas ubi quae senti...,ca
282395,282395,282395,485530,3885,(субб): Dies ordin[arius] atque tristis,1809-07-15,1.0,1776-01-24,33.0,субб dies ordin arius atque tristis \n,ca


In [44]:
ca_data.shape

(7, 11)

In [45]:
# Тексты на вьетнамском языке
vi_data = data[data['language'] == 'vi']
vi_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
154373,154373,154373,248413,324,— -5˚C.,1941-10-10,1.0,1868-10-28,73.0,— ˚ c \n,vi
391166,391166,391166,695300,201,"20, ср.<еда>. В Longchêne(s) x.",1938-07-20,1.0,1886-05-28,52.0,ср еда longchêne s x \n,vi
391186,391186,391186,695341,201,"14, ср.<еда>. В Longchêne(s) x.",1938-09-14,1.0,1886-05-28,52.0,ср еда longchêne s x \n,vi


In [46]:
vi_data.shape

(3, 11)

In [47]:
# Тексты на словенском языке
sl_data = data[data['language'] == 'sl']
sl_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
165963,165963,165963,267854,1049,Сентября 20. Я hodil q Оbedni. У меня bыl учиt...,1887-10-02,1.0,1854-01-01,33.0,сентябрь hodil q оbedni bыl учиtel segodнi seg...,sl
199714,199714,199714,324183,963,23. У Nazar Aga. Был Леон.,1903-11-05,1.0,1832-01-01,71.0,nazar aga леон \n,sl
242527,242527,242527,407078,1847,Napoli.,1910-01-17,1.0,1860-01-01,50.0,napoli \n,sl
242528,242528,242528,407079,1847,Napoli.,1910-01-19,1.0,1860-01-01,50.0,napoli \n,sl
423768,423768,423768,756574,7077,18/IV № 13 — мое на 617,1942-04-18,0.0,1907-10-13,35.0,iv № — \n,sl


In [48]:
sl_data.shape

(6, 11)

In [50]:
# Тексты на норвежском языке
no_data = data[data['language'] == 'no']
no_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
199747,199747,199747,324216,963,5. Merenville с женою.,1903-12-18,1.0,1832-01-01,71.0,merenville жена \n,no
199792,199792,199792,324261,963,23. Ditil Dr Электрич[?]Merenvile.,1904-02-05,1.0,1832-01-01,72.0,ditil dr электрич merenvile \n,no
199799,199799,199799,324268,963,2. Ditil Электр[?] Dr. Batal de fleur. Были Me...,1904-02-15,1.0,1832-01-01,72.0,ditil электр dr batal de fleur merenville л ля...,no
199803,199803,199803,324273,963,8. Фигеро. Merenville.,1904-02-21,1.0,1832-01-01,72.0,фигеро merenville \n,no
199814,199814,199814,324284,963,"Февраля 19. Были Erb, Ditile, Fiegero.",1904-03-03,1.0,1832-01-01,72.0,февраль erb ditile fiegero \n,no


In [51]:
no_data.shape

(9, 11)

In [52]:
# Тексты на датском языке
da_data = data[data['language'] == 'da']
da_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
199816,199816,199816,324286,963,"21. Merenvilli были, а я у них.",1904-03-05,1.0,1832-01-01,72.0,merenvilli \n,da
242201,242201,242201,406750,1847,Helsingfors.,1901-07-18,1.0,1860-01-01,41.0,helsingfors \n,da
242202,242202,242202,406751,1847,Helsingfors.,1901-07-19,1.0,1860-01-01,41.0,helsingfors \n,da


In [53]:
# Тексты на голландском языке
nl_data = data[data['language'] == 'nl']
nl_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
199905,199905,199905,324378,963,"10. 2 — — — — — — — — — ,был Basch. \nЛазарь,...",1904-06-23,1.0,1832-01-01,72.0,— — — — — — — — — basch лазарь аннет мери на...,nl
242047,242047,242047,406596,1847,Genève.,1897-06-14,1.0,1860-01-01,37.0,genève \n,nl
252822,252822,252822,427267,840,Mementо...,1882-10-24,1.0,1815-09-22,67.0,mementо \n,nl
302790,302790,302790,520042,1313,"Berlin, Thiergarten. Осень.",1910-11-07,1.0,1891-01-01,19.0,berlin thiergarten осень \n,nl


In [54]:
# Тексты на валлийском языке
cy_data = data[data['language'] == 'cy']
cy_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
199983,199983,199983,324458,963,12. Myanac у меня.,1904-09-25,1.0,1832-01-01,72.0,myanac \n,cy


In [55]:
# Тексты на хорватском языке
hr_data = data[data['language'] == 'hr']
hr_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
220623,220623,220623,368521,1990,8-9-I-49 г.,1949-01-08,1.0,1898-01-01,51.0,i г \n,hr
341216,341216,341216,576161,3798,Mon journal\n19. \n12-I-42.—22-I-42.,1942-01-12,1.0,1921-12-19,21.0,mon journal i — i \n,hr


In [56]:
# Тексты на африкаанс языке
af_data = data[data['language'] == 'af']
af_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
220791,220791,220791,368689,984,(пн.): Dies ordinaries et tristes. Ездили по м...,1991-11-04,0.0,1977-08-25,14.0,пн dies ordinaries et tristes ездить магазин \n,af
282311,282311,282311,485440,3885,(ср): Dies ordin[arius],1809-03-22,1.0,1776-01-24,33.0,ср dies ordin arius \n,af
282318,282318,282318,485447,3885,(ср): Dies ordin[arius],1809-03-29,1.0,1776-01-24,33.0,ср dies ordin arius \n,af
282348,282348,282348,485479,3885,(воскр): Dies ordin[arius],1809-05-07,1.0,1776-01-24,33.0,воскра dies ordin arius \n,af
282394,282394,282394,485529,3885,(пятн): Dies ordin[arius] atque trist[is],1809-07-14,1.0,1776-01-24,33.0,пятна dies ordin arius atque trist is \n,af


In [57]:
# Тексты на индонезийском языке
id_data = data[data['language'] == 'id']
id_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
223158,223158,223158,376506,1181,"Das Jangubuch\n""",1910-02-14,0.0,1892-10-11,18.0,das jangubuch \n,id


In [58]:
# Тексты на тагальском языке
tl_data = data[data['language'] == 'tl']
tl_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
242164,242164,242164,406713,1847,Lugano.,1900-08-17,1.0,1860-01-01,40.0,lugano \n,tl
242165,242165,242165,406714,1847,Milano.,1900-09-01,1.0,1860-01-01,40.0,milano \n,tl


In [59]:
# Тексты на шведском языке
sv_data = data[data['language'] == 'sv']
sv_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
242203,242203,242203,406752,1847,Stokholm.,1901-07-21,1.0,1860-01-01,41.0,stokholm \n,sv
242205,242205,242205,406754,1847,Stokholm.,1901-07-23,1.0,1860-01-01,41.0,stokholm \n,sv
242207,242207,242207,406756,1847,Stokholm.,1901-08-16,1.0,1860-01-01,41.0,stokholm \n,sv
275899,275899,275899,475180,3337,Я играю еще в Rock Fall,1993-01-11,1.0,1985-12-12,8.0,играть rock fall \n,sv


In [61]:
# Тексты на греческом языке
el_data = data[data['language'] == 'el']
el_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
246076,246076,246076,413831,831,"Τρίώδίον\n""",1850-02-24,1.0,1817-01-01,33.0,τρίώδίον \n,el
246155,246155,246155,413996,831,"ΣΕΠΤΕΜΒΡΙΟΣ\n""",1850-09-14,1.0,1817-01-01,33.0,σεπτεμβριος \n,el


In [62]:
# Тексты на сомалийском языке
so_data = data[data['language'] == 'so']
so_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
360097,360097,360097,613370,2304,11 XI — 53 г.,1953-11-11,0.0,1939-10-06,14.0,xi — г \n,so
373266,373266,373266,634802,5200,18/II.31 г.,1931-02-18,1.0,1907-06-11,24.0,ii г \n,so
423068,423068,423068,754952,7636,"18 Д. — 13/XI, 19 д. — 14/XI; 20 д. 15/XI, 21 ...",1935-11-13,1.0,1892-08-21,43.0,д — xi д — xi д xi д — xi \n,so
423762,423762,423762,756568,7077,16/II 42 — № 6 — о том же.,1942-02-16,0.0,1907-10-13,35.0,ii — № — \n,so
423787,423787,423787,756594,7077,№ 50 22/X,1942-10-22,0.0,1907-10-13,35.0,№ x \n,so


In [64]:
# Тексты на португальском языке
pt_data = data[data['language'] == 'pt']
pt_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
397377,397377,397377,710003,2174,18 июля 1937 г.Embras de richesse!...,1937-07-18,1.0,1862-04-25,75.0,июль г embras de richesse \n,pt


In [65]:
# Тексты на финском языке
fi_data = data[data['language'] == 'fi']
fi_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
423734,423734,423734,756538,7077,16/VIII -,1941-08-16,0.0,1907-10-13,34.0,viii \n,fi


In [67]:
# Тексты на словацком языке
sk_data = data[data['language'] == 'sk']
sk_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
423773,423773,423773,756579,7077,13/V № 18 на 617 -,1942-05-13,0.0,1907-10-13,35.0,v № \n,sk


In [68]:
# Тексты на украинском языке
uk_data = data[data['language'] == 'uk']
uk_data.head(5)

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
1333,1333,1333,2530,10,Днем мама и Женя.,1912-09-14,1.0,1880-11-28,32.0,день мама женя \n,uk
1468,1468,1468,2665,10,Днем — мама у меня.,1913-02-21,1.0,1880-11-28,33.0,день — мама \n,uk
3314,3314,3314,5870,14,Весь день очень холодно. Взята Либава.,1945-05-10,1.0,1909-05-29,36.0,весь день очень холодно взять либава \n,uk
4454,4454,4454,7308,18,"Звонила Оленька, пригласила нас завтра к себе.",1937-12-20,0.0,1893-11-02,44.0,звонить оленька приглашать завтра \n,uk
4776,4776,4776,7630,18,Были Виленкин и Оля. Миша читал им фрагменты. ...,1939-06-22,0.0,1893-11-02,46.0,виленкин оля миша читать фрагмент видеть нрави...,uk


Удаляем тексты с вкраплениями на другом языке.

In [45]:
start_shape = data.shape
print(start_shape)

(437852, 11)


In [46]:
values_to_remove = ['de', 'en', 'fr', 'error', 'pl', 'ro', 'it', "('undefined', nan)", 'ca', 'vi', 'sl', 'no', 'da', 'nl', 'cy', 'hr', 'af', 'id', 'tl', 'sv', 'el', 'so', 'pt', 'fi', 'sk']

data = data[~data['language'].isin(values_to_remove)]

In [47]:
print(f'Было удалено {start_shape[0] - data.shape[0]} строки.')

Было удалено 562 строки.


Удаляем отдельные буквы и их сочетания, поскольку многие, например, римские цифры были индикатором для отнесения текста к английскому языку.

In [48]:
def remove_single_letters(text):
  """Функция для удаления отдельно стоящих букв"""
  return re.sub(r'\b\w\b', '', text)

In [49]:
tqdm.pandas(desc="Processing rows")
data['text_preprocessing'] = data['text_preprocessing'].progress_apply(remove_single_letters)

Processing rows:   0%|          | 0/437290 [00:00<?, ?it/s]

In [76]:
def remove_non_cyrillic_words(text):
  """Функция для удаления слов с иностранными буквами кроме кириллицы"""
  return re.sub(r'\b(?![А-Яа-яЁё])\w+\b', '', text)

In [77]:
tqdm.pandas(desc="Processing rows")
data['text_preprocessing'] = data['text_preprocessing'].progress_apply(remove_non_cyrillic_words)

Processing rows:   0%|          | 0/437290 [00:00<?, ?it/s]

In [84]:
def remove_roman_numerals(text):
  """Функция удаления римских цифр"""
  return re.sub(r'\b[ivxlcdmIVXLCDMххiii]+\b', '', text)

In [85]:
tqdm.pandas(desc="Processing rows")
data['text_preprocessing'] = data['text_preprocessing'].progress_apply(remove_roman_numerals)

Processing rows:   0%|          | 0/437290 [00:00<?, ?it/s]

In [87]:
# Вывод строк, где есть слова с буквой 'i'
rows_with_i = data[data['text_preprocessing'].str.contains('i')]

rows_with_i

,Unnamed: 0.1,Unnamed: 0,id,diary,text,date,sex,birth,age,text_preprocessing,language
2143,2143,2143,4699,13,"Суббота. Утро очень ясное, опять с каким‑то ос...",1916-08-26,1.0,1867-04-06,49.0,суббота утро очень ясный ‑ осенний оттенок сде...,ru
6463,6463,6463,9523,24,В Минводах было спокойно и даже не видно следо...,1941-11-03,1.0,1923-03-01,18.0,минводы спокойно видно след недавний бомбардир...,ru
6561,6561,6561,9621,24,Сегодня разругался с хозяйкой — высказал откро...,1943-02-27,1.0,1923-03-01,20.0,сегодня разругаться хозяйка — высказывать от...,ru
6568,6568,6568,9628,24,"14 марта, после ночевки на Пролетарской в семь...",1943-03-17,1.0,1923-03-01,20.0,марта ночевка пролетарский семь час утро пойти...,ru
6693,6693,6693,9753,24,Несколько раз вчера просил Запрягайло отпустит...,1944-02-07,1.0,1923-03-01,21.0,несколько вчера просить запрягайло отпускать д...,ru
...,...,...,...,...,...,...,...,...,...,...,...
432626,432626,432626,772999,1053,3 августаII конференция ВЛКСМ Куйбышевского ра...,1937-08-03,0.0,1920-12-24,17.0,августаii конференция влксм куйбышевский район...,ru
433304,433304,433304,774282,3380,17 июняВражеские части заняли плацдарм на лево...,1944-06-17,1.0,1916-11-25,28.0,июнявражеский часть занимать плацдарм левый бе...,ru
433305,433305,433305,774283,3380,"18 июняСладок утренний сон, но вот нашего слух...",1944-06-18,1.0,1916-11-25,28.0,июнясладкий утренний сон наш слух касаться едв...,ru
434720,434720,434720,779176,9925,8 августа. Воскресенье. Вчера ездили на пригор...,1954-08-08,1.0,1934-02-04,20.0,август воскресение вчера ездить пригородный по...,ru


In [91]:
def remove_word_with_i(text):
  """Функция для удаления слов с буквой 'i'"""
  words = text.split()
  return ' '.join([word for word in words if 'i' not in word.lower()])

In [92]:
tqdm.pandas(desc="Processing rows")
data['text_preprocessing'] = data['text_preprocessing'].progress_apply(remove_word_with_i)

Processing rows:   0%|          | 0/437290 [00:00<?, ?it/s]

In [97]:
del data['Unnamed: 0.1']
del data['Unnamed: 0']

In [104]:
def remove_special_characters(text):
  """Функция для повторного удаления тире, кавычек, символов новой строки и табуляции"""
  return re.sub(r'[-—\'"“”„‘’«»\n\t]', '', text)

In [105]:
tqdm.pandas(desc="Processing rows")
data['text_preprocessing'] = data['text_preprocessing'].progress_apply(remove_special_characters)

Processing rows:   0%|          | 0/437290 [00:00<?, ?it/s]

In [107]:
def remove_multiple_spaces(text):
  """Функция удаления множественных пробелов"""
  return re.sub(r'\s+', ' ', text, flags=re.I)

In [108]:
tqdm.pandas(desc="Processing rows")
data['text_preprocessing'] = data['text_preprocessing'].progress_apply(remove_multiple_spaces)

Processing rows:   0%|          | 0/437290 [00:00<?, ?it/s]

In [113]:
data.shape

(437290, 9)

In [114]:
data.dropna()

,id,diary,text,date,sex,birth,age,text_preprocessing,language
0,693,3,Сейчас бьет на башенных часах 12. Полночь. Жен...,1932-09-28,1.0,1890-05-25,42.0,бить башенный часы полночь жена спать ребенок ...,ru
1,694,3,"Читал Шаляпина — воспоминания. Оказывается, ис...",1932-10-06,1.0,1890-05-25,42.0,читать шаляпин воспоминание оказываться истори...,ru
2,695,3,При социализме заводы суть храмы и центры чело...,1932-10-20,1.0,1890-05-25,42.0,социализм завод храм центр человеческий житие ...,ru
3,696,3,Завтра с утра — в Москву. Что-то меня там ждет...,1932-10-30,1.0,1890-05-25,42.0,завтра утро москва ждать каково настроение шир...,ru
4,697,3,"В Берлине, в полпредстве, оказался Горький (пр...",1932-11-01,1.0,1890-05-25,42.0,берлин полпредство оказываться горький проезд ...,ru
...,...,...,...,...,...,...,...,...,...
437847,785544,9939,"21 сентября (понедельник), 1970 год\nКак мне х...",1970-09-21,1.0,1940-06-12,30.0,сентябрь понедельник год вчера тяжело плохо се...,ru
437848,785545,9939,"22 сентября (вторник), 1970 годПисать сегодня ...",1970-09-22,1.0,1940-06-12,30.0,сентябрь вторник годписывать сегодня поговорит...,ru
437849,785546,9939,"23 сентября (среда), 1970 год\nНу вот прошел и...",1970-09-23,1.0,1940-06-12,30.0,сентябрь среда год проходить сегодняшний день ...,ru
437850,785547,9939,"24 сентября (четверг), 1970 годМы были вместе ...",1970-09-24,1.0,1940-06-12,30.0,сентябрь четверг годма вместе вечер ничто стан...,ru


In [116]:
del data['language']

In [118]:
data.head(5)

,id,diary,text,date,sex,birth,age,text_preprocessing
0,693,3,Сейчас бьет на башенных часах 12. Полночь. Жен...,1932-09-28,1.0,1890-05-25,42.0,бить башенный часы полночь жена спать ребенок ...
1,694,3,"Читал Шаляпина — воспоминания. Оказывается, ис...",1932-10-06,1.0,1890-05-25,42.0,читать шаляпин воспоминание оказываться истори...
2,695,3,При социализме заводы суть храмы и центры чело...,1932-10-20,1.0,1890-05-25,42.0,социализм завод храм центр человеческий житие ...
3,696,3,Завтра с утра — в Москву. Что-то меня там ждет...,1932-10-30,1.0,1890-05-25,42.0,завтра утро москва ждать каково настроение шир...
4,697,3,"В Берлине, в полпредстве, оказался Горький (пр...",1932-11-01,1.0,1890-05-25,42.0,берлин полпредство оказываться горький проезд ...


## **Работа с аномальными значениями**

In [120]:
data['age'].describe()

count    437290.000000
mean         42.742974
std          17.665688
min         -44.000000
25%          28.000000
50%          41.000000
75%          57.000000
max         136.000000
Name: age, dtype: float64

In [123]:
# Подсчет строк с аномальными значениями
anomalous_rows_count = data[(data['age'] > 100) | (data['age'] < 0)].shape[0]
print(f'Количество строк с аномальными значениями: {anomalous_rows_count}.')

Количество строк с аномальными значениями: 60.


In [126]:
# Удаление строк с аномальными значениями
data = data[(data['age'] <= 100) & (data['age'] >= 0)]

In [127]:
data.shape

(437230, 8)

In [129]:
data.to_csv('data_preprocessing_final.csv')

## **Количественный анализ датасета**

In [133]:
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [134]:
# Пример функции для токенизации текста и подсчета токенов
def count_tokens(text):
    tokens = word_tokenize(text)
    return len(tokens)

data['token_count'] = data['text_preprocessing'].apply(count_tokens)

<ipython-input-134-5aacd5d3876c>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['token_count'] = data['text_preprocessing'].apply(count_tokens)


In [136]:
data['token_count'].sum()

39544413

In [139]:
data['token_count'].describe()

count    437230.000000
mean         90.443046
std         148.834047
min           0.000000
25%          20.000000
50%          46.000000
75%         101.000000
max        9855.000000
Name: token_count, dtype: float64

In [141]:
data['diary'].nunique()

2139